Przed oddaniem zadania upewnij się, że wszystko działa poprawnie.
**Uruchom ponownie kernel** (z paska menu: Kernel$\rightarrow$Restart) a następnie
**wykonaj wszystkie komórki** (z paska menu: Cell$\rightarrow$Run All).

Upewnij się, że wypełniłeś wszystkie pola `TU WPISZ KOD` lub `TU WPISZ ODPOWIEDŹ`, oraz
że podałeś swoje imię i nazwisko poniżej:

In [ ]:
NAME = ""

---

# BYOL
W niniejszym zeszycie skupimy się na metodzie *Bootsrap Your Own Latent* (BYOL) [(Grill et al., 2020)](https://arxiv.org/abs/2006.07733). **Należy się dokładnie zapoznać z ideą i zasadą działania BYOL** (wykorzystując materiały z wykładu oraz publikacje, bądź inne materiały dostępne w sieci). 

W niniejszym zeszycie należy wykonać zadania z zakresu implementacji metody BYOL oraz badania jej hiperparametrów. Model oraz trenowanie zaimplementowano z wykorzystaniem `pytorch_lightning`. Żeby zrozumieć dobrze implementacje, należy zapoznać się z klasą bazową `SSLBase` i ewaluacją na zadaniu docelowym, która została w tej klasie zaimplementowana.

In [ ]:
%load_ext tensorboard

In [ ]:
import copy

import torch
from lightning_fabric import seed_everything
from lightning.pytorch.loggers import TensorBoardLogger
from lightning import Trainer
from torch import nn, Tensor
from torch.nn import functional as F

from src.data import VisionDatamodule
from src.ssl_base import SSLBase
from src.networks import SmallConvnet, MLP
from src.augmentations import get_default_aug

# Zbiór danych
Do uczenia BYOL wykorzystamy wypróbkowany podzbiór zbioru MNIST. Warto zaznaczyć, że jest to wybór podyktowany tylko i wyłącznie ograniczeniami w zasobach obliczeniowych. W rzeczywistości, aby otrzymać konkluzywne i rzetelne wyniki, należałoby skorzystać co najmniej ze zbioru `CIFAR-10` oraz znacznie większego modelu kodera, np. `ResNet`. Jednak do celów dydaktycznych skorzystamy z mniejszego zbioru oraz odpowiednio mniejszego kodera.

In [ ]:
# define parameters
DATA_DIR = "./data"
OUT_DIR = "./data/output/byol"
BATCH_SIZE = 256
NUM_WORKERS = 0
NUM_SAMPLES_PER_CLASS = 300

In [ ]:
datamodule = VisionDatamodule(
    root_dir=DATA_DIR,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    num_samples_per_class=NUM_SAMPLES_PER_CLASS,
)
datamodule.prepare_data()
datamodule.setup()

# Zadanie 3.1 Implementacja modelu BYOL (2.5 pkt)
W poniższej komórce znajduje się częściowa implementacja modelu `BYOL`. Uzupełnij brakujące implementacje funkcji:
* `copy_and_freeze_module` (0.5 pkt) - kopiuje parametry sieci oraz "zamraża" te skopiowane (ustawia brak liczenia gradientu dla skopiowanych parametrów)
* `byol_loss` (1.0 pkt) - funkcja straty modelu BYOL (zgodnie z oryginalną publikacją)
  $$ \mathcal{L}_{\theta, \zeta} = \lVert \bar{q_{\theta}}(z_{\theta}) - \bar{z'_{\zeta}} \rVert_2^2 $$
  gdzie $\bar{q_{\theta}}(z_{\theta})$ - wektor wyjściowy z predyktora gałęzi `ONLINE`, $\bar{z'_{\zeta}}$ - wektor wyjściowy z projektora gałęzi `TARGET`
* `update_target_network` (1.0 pkt) - aktualizuje parametry kodera oraz projektora sieci `TARGET`, ustawia je jako średnia krocząca sieci `ONLINE`

Ponadto:
* Przeanalizuj dokładnie pozostałe elementy implementacji.
* Zastanów się, czy wszystkie parametry modelu podlegają uczeniu?
* Zastanów dlaczego w funkcji `forward` dodatkowo liczone jest `q_sym` oraz `z_prim_sym`, czy bez tego model będzie nadal działał poprawnie?
* Uruchom uczenie i przeanalizuj wyniki.

In [ ]:
class BYOLModel(SSLBase):
    def __init__(
        self,
        learning_rate: float,
        weight_decay: float,
        tau: float,
        out_channels: int = 10,
    ):
        super().__init__(
            learning_rate=learning_rate,
            weight_decay=weight_decay,
            out_channels=out_channels,
        )

        # Initialize online network
        self.online_encoder = SmallConvnet()
        self.online_projector = MLP(84, 84, 84, plain_last=False)
        self.online_predictor = MLP(84, 84, 84, plain_last=True)
        self.online_net = nn.Sequential(
            self.online_encoder,
            self.online_projector,
            self.online_predictor,
        )

        # Initialize target network with frozen weights
        self.target_encoder = self.copy_and_freeze_module(self.online_encoder)
        self.target_projector = self.copy_and_freeze_module(self.online_projector)
        self.target_net = nn.Sequential(self.target_encoder, self.target_projector)

        # Initialize augmentations
        self.aug_1 = get_default_aug()
        self.aug_2 = get_default_aug()

        self.tau = tau

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        t = self.aug_1(x)
        t_prim = self.aug_2(x)

        q = self.online_net(t)
        q_sym = self.online_net(t_prim)

        with torch.no_grad():
            z_prim = self.target_net(t_prim)
            z_prim_sym = self.target_net(t)

        q = torch.cat([q, q_sym], dim=0)
        z_prim = torch.cat([z_prim, z_prim_sym], dim=0)

        return q, z_prim

    def forward_repr(self, x: Tensor) -> Tensor:
        return self.online_encoder(x)

    def training_step(self, batch: Tensor, batch_idx: int) -> Tensor:
        x, _ = batch
        q, z_prim = self.forward(x)
        loss = self.byol_loss(q=q, z_prim=z_prim)

        self.log("train/loss", loss, prog_bar=True)

        return loss

    def byol_loss(self, q: Tensor, z_prim: Tensor) -> Tensor:
        # TU WPISZ KOD
        raise NotImplementedError()

    def on_train_epoch_end(self) -> None:
        super().on_train_epoch_end()
        self.update_target_network()

    @torch.no_grad()
    def update_target_network(self) -> None:
        # TU WPISZ KOD
        raise NotImplementedError()

    @staticmethod
    def copy_and_freeze_module(model: nn.Module) -> nn.Module:
        # TU WPISZ KOD
        raise NotImplementedError()

In [ ]:
%tensorboard --logdir "./data/output/byol"

In [ ]:
# define hyperparameters
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
TAU = 0.99
EPOCHS = 200
ACCELERATOR = "cpu"  # change to CUDA, if want to train on GPU

seed_everything(42)
model = BYOLModel(
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    tau=TAU,
)
logger = TensorBoardLogger(save_dir=OUT_DIR, default_hp_metric=False)
trainer = Trainer(
    default_root_dir=OUT_DIR,
    max_epochs=EPOCHS,
    logger=logger,
    accelerator=ACCELERATOR,
    num_sanity_val_steps=0,
    log_every_n_steps=10,
)

trainer.fit(model, datamodule)
trainer.test(model, datamodule)

# Zadanie 3.2 Znaczenie projektora (0.5 pkt)
* Zmodyfikuj klasę modelu BYOL, tak aby nie zawierał on projektora (zarówno w sieci `ONLINE` jak i `TARGET`) i sprawdzić jak różnią się otrzymane wyniki względem poprzedniego eksperymentu
* Zinterpretuj otrzymane wyniki, jaka jest rola projektora w tym modelu?

In [ ]:
class BYOLWithoutProjectorModel(SSLBase):
    # TU WPISZ KOD
    raise NotImplementedError()

In [ ]:
# define hyperparameters
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
TAU = 0.99
EPOCHS = 200
ACCELERATOR = "cpu"  # change to CUDA, if want to train on GPU

seed_everything(42)
model = BYOLWithoutProjectorModel(
    learning_rate=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY,
    tau=TAU,
)
logger = TensorBoardLogger(save_dir=OUT_DIR, default_hp_metric=False)
trainer = Trainer(
    default_root_dir=OUT_DIR,
    max_epochs=EPOCHS,
    logger=logger,
    accelerator=ACCELERATOR,
    num_sanity_val_steps=0,
    log_every_n_steps=10,
)

trainer.fit(model, datamodule)
trainer.test(model, datamodule)

# Zadanie 3.3 Badanie parametru `tau` w EMA (1 pkt)
* Sprawdź jakie wyniki model osiąga dla różnych wartości parametru `tau`, w szczególności przebadaj wartości krańcowe `(0.0, 1.0)`
* Idelanie będzie wykonać 3-5 powtórzeń dla każdej wartości parametru, ale jeśli ograniczają Cię zasoby wystarczy 1
* Do wyznaczenia metryk wykorzystaj metodę `trainer.test(...)` i przygotuj wykres na podstawie otrzymanych wyników
* Pamiętaj, aby badać pierwszą wersję modelu z projektorem, a z każdym kolejnym uruchominiem uczenia ustawiaj na nowo seed

In [ ]:
# TU WPISZ KOD
raise NotImplementedError()